In [1]:
import MARBL_tools # Note: I softlinked $MARBL/MARBL_tools into the defaults directory
                   #       I also needed to update some of the python to grab more data from the JSON file
    
import yaml
import pandas as pd
import numpy as np

%load_ext watermark
%watermark -d -iv -m -g -h

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 3.10.0-1127.18.2.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 72
Architecture: 64bit

Hostname: crhtc47

Git hash: c00eac42969559bb25f046f2b8cae59d489a826d

pandas: 1.2.4
numpy : 1.20.2
yaml  : 5.4.1



In [2]:
# I think the easiest way to have consistency in units is to create a function here
def _unit_conversion(old_units):
    new_units = old_units.replace("^2", "$^2$").replace("^3", "$^3$").replace("1/day", "d$^{-1}$")
    return new_units

In [3]:
with open('MARBL_settings.yaml', encoding='utf-8') as file_in:
    config_dict = yaml.safe_load(file_in)
MARBL_settings = MARBL_tools.MARBL_settings_class('defaults/json/settings_cesm2.1.json',
                                                  config_dict['defaults']['saved_state_source'],
                                                  config_dict['defaults']['grid']
                                                 )
table_dict = {}
table_dict['Parameter'] = []
table_dict['Value'] = []
table_dict['Units'] = []
table_dict['Description'] = []
for varname in config_dict['Parameters']:
    # If table value isn't in MARBL_settings.yaml, we want to get it from MARBL_settings.settings_dict[varname]
    # but some variables are modified such that we need to look at MARBL_settings._settings['general_parms'][varname]
    if all([key in config_dict['Parameters'][varname] for key in ['label', 'units', 'description', 'value']]):
        param_value = config_dict['Parameters'][varname]['value']
        # fallback_dict shouldn't be used, but if it is I don't want to rely on garbage values fom last varname
        fallback_dict = {'units': config_dict['Parameters'][varname]['units'],
                         'longname': config_dict['Parameters'][varname]['description'],
                        }
    else:
        try:
            fallback_dict = MARBL_settings.settings_dict[varname]['attrs']
            param_value = MARBL_settings.settings_dict[varname]['value']
        except:
            fallback_dict = MARBL_settings._settings['general_parms'][varname]
            try:
                param_value = MARBL_settings._settings['general_parms'][varname]['default_value']['default']
            except:
                param_value = MARBL_settings._settings['general_parms'][varname]['default_value']

    # Parameters column contains variable name unless a different label was specified
    label = config_dict['Parameters'][varname].get('label', varname)
    table_dict['Parameter'].append(label)

    # Need a way to standardize units
    units = config_dict['Parameters'][varname].get('units', fallback_dict['units'])
    table_dict['Units'].append(_unit_conversion(units))

    # Description column contains variable longname unless a different description was specified
    desc = config_dict['Parameters'][varname].get('description', fallback_dict['longname'])
    if 'description_suffix' in config_dict['Parameters'][varname]:
        desc = desc + f" {config_dict['Parameters'][varname]['description_suffix']}"
    table_dict['Description'].append(desc)

    # value comes from parsing the JSON file
    
    # try / except block in case the variable is not numeric
    try:
        if param_value != 0 and np.abs(param_value < 0.01):
            param_value = f"{param_value:.1e}"
    except:
        pass
    table_dict['Value'].append(param_value)

# Make table
pd.set_option("max_colwidth", 256)
pd.DataFrame(table_dict)

,Parameter,Value,Units,Description
0,$\mu_{ref}$,5,d$^{-1}$,Maximum C-spec growth rate at Tref for small phytoplankton and diatoms
1,"$\mu_{ref,\mathrm{diaz}}$",2.5,d$^{-1}$,Maximum C-spec growth rate at Tref for diazotrophs
2,$Q_{10}$,1.7,unitless,Q10 temperature coefficient
3,$T_{ref}$,30,$^\circ$C,Reference temperature
4,"$\theta^{N}_{max,\mathrm{sp}}$",2.5,mg Chl / mmol,Maximum $\theta^N$ (Chl / N) for small phytoplankton
5,"$\theta^{N}_{max,\mathrm{diat}}$",4,mg Chl / mmol,Maximum $\theta^N$ (Chl / N) for diatoms
6,"$\theta^{N}_{max,\mathrm{diaz}}$",2.5,mg Chl / mmol,Maximum $\theta^N$ (Chl / N) for diazotrophs
7,$\alpha^{Chl}_{\mathrm{sp}}$,0.39,mmol m$^2$/(mg Chl W day),Initial slope of P\_I curve (GD98) for small phytoplankton
8,$\alpha^{Chl}_{\mathrm{diat}}$,0.28,mmol m$^2$/(mg Chl W day),Initial slope of P\_I curve (GD98) for diatoms
9,$\alpha^{Chl}_{\mathrm{diaz}}$,0.39,mmol m$^2$/(mg Chl W day),Initial slope of P\_I curve (GD98) for diazotrophs


In [4]:
print(pd.DataFrame(table_dict).to_latex(index=False, escape=False))

\begin{tabular}{llll}
\toprule
                       Parameter &                                 Value &                     Units &                                                            Description \\
\midrule
                     $\mu_{ref}$ &                                     5 &                  d$^{-1}$ & Maximum C-spec growth rate at Tref for small phytoplankton and diatoms \\
       $\mu_{ref,\mathrm{diaz}}$ &                                   2.5 &                  d$^{-1}$ &                     Maximum C-spec growth rate at Tref for diazotrophs \\
                        $Q_{10}$ &                                   1.7 &                  unitless &                                            Q10 temperature coefficient \\
                       $T_{ref}$ &                                    30 &                 $^\circ$C &                                                  Reference temperature \\
  $\theta^{N}_{max,\mathrm{sp}}$ &                                   2.5 & 